In [ ]:
# weather script
# aug 4th
# open meteo api

In [5]:
! pip install openmeteo-requests
! pip install requests-cache retry-requests numpy pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 669.9/669.9 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.6/635.6 kB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 53.7 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.11.0
    Uninstalling typing_extensions-4.11.0:
      Successfully uninstalled typing_extensions-4.11.0
  Attempting uninstall: attrs
    Found existing installation: attrs 23.1.0
    Uninstalling attrs-23.1.0:
      Successfully uninstalled attrs-23.1.0


In [51]:
import openmeteo_requests

import pandas as pd
import os
import requests_cache
from retry_requests import retry
from datetime import datetime, timezone, timedelta
from zoneinfo import ZoneInfo  # to recognize the place by its coordinates

In [ ]:
# setup the api client with cache and retry on error
# from the api code

cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

In [ ]:
# weather variables
# single location

url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": 40.74,
	"longitude": -73.87,
	"daily": ["weather_code", "sunrise", "sunset", "daylight_duration", "sunshine_duration", "precipitation_sum", "precipitation_probability_max", "apparent_temperature_max", "apparent_temperature_min", "temperature_2m_max", "temperature_2m_min", "precipitation_hours"],
	"hourly": ["temperature_2m", "apparent_temperature", "precipitation", "cloud_cover", "is_day", "sunshine_duration"],
	"current": ["temperature_2m", "apparent_temperature", "is_day", "precipitation", "weather_code"],
	"timezone": "auto",
	"past_days": 7,
}
responses = openmeteo.weather_api(url, params=params)

In [44]:
# set up

weather_code_map = {
    0: "Clear sky",
    1: "Mainly clear",
    2: "Partly cloudy",
    3: "Overcast",
    45: "Fog",
    48: "Depositing rime fog",
    51: "Light drizzle",
    53: "Moderate drizzle",
    55: "Dense drizzle",
    56: "Light freezing drizzle",
    57: "Dense freezing drizzle",
    61: "Slight rain",
    63: "Moderate rain",
    65: "Heavy rain",
    66: "Light freezing rain",
    67: "Heavy freezing rain",
    71: "Slight snow fall",
    73: "Moderate snow fall",
    75: "Heavy snow fall",
    77: "Snow grains",
    80: "Slight rain showers",
    81: "Moderate rain showers",
    82: "Violent rain showers",
    85: "Slight snow showers",
    86: "Heavy snow showers",
    95: "Thunderstorm (slight or moderate)",
    96: "Thunderstorm with slight hail",
    99: "Thunderstorm with heavy hail"
}

weather_code_map[51]

'Light drizzle'

In [59]:
# data of one location
# in case we want to request more than one location, it would be the same structure

response = responses[0]

tz_name = response.Timezone().decode("utf-8")
local_tz = ZoneInfo(tz_name)

latitude = f'{response.Latitude():.2f}'
longitude = f'{response.Longitude():.2f}'
elevation = f'{response.Elevation()}'
timezone = f'{response.TimezoneAbbreviation().decode('utf-8')}'

print(f"Place: {tz_name}")
print(f"Coordinates: {latitude} {longitude}")
print(f"Elevation: {elevation}m asl") # asl = above sea level
print(f"Timezone: {timezone}")
# print(f"Timezone difference to GMT+0: {response.UtcOffsetSeconds() / 3600:.0f}h")


Place: America/New_York
Coordinates: 40.74 -73.87
Elevation: 18.0m asl
Timezone: GMT-4


In [ ]:
# process current data
# the order of variables needs to be the same as requested.

#getting the data
current = response.Current()
current_temperature_2m = current.Variables(0).Value()
current_apparent_temperature = current.Variables(1).Value()
current_is_day = current.Variables(2).Value() # day = 1 night = 0
current_precipitation = current.Variables(3).Value()
current_weather_code = current.Variables(4).Value()

#formating
readable_time = datetime.fromtimestamp(current.Time(), tz=timezone.utc)
formatted_date = readable_time.strftime("%Y-%m-%d")
formatted_time = readable_time.strftime("%H:%M UTC")
# current_time_inlocal = response.UtcOffsetSeconds()
# local_tz = timezone(timedelta(seconds=current_time_inlocal))
readable_time_inlocal = datetime.fromtimestamp(current.Time(), tz=local_tz)
formatted_time_inlocal = readable_time_inlocal.strftime("%H:%M %Z")
is_day = "day" if current_is_day == 1 else "night"
weather_description = weather_code_map.get(int(current_weather_code), "Unknown")

#printing
print(f"{tz_name}")
print(f"Coordinates: {response.Latitude():.2f}°N {response.Longitude():.2f}°E")
print(f"Elevation: {response.Elevation()}m asl") # asl = above sea level
print(f"Timezone: {response.TimezoneAbbreviation().decode('utf-8')}")

print(f"\nCurrent date: {formatted_date}")
print(f"Current local time: {formatted_time_inlocal}")
print(f"Current universal time: {formatted_time}")
print(f"Current temperature_2m: {current_temperature_2m:.2f}°C")
print(f"Current apparent_temperature: {current_apparent_temperature:.2f}°C")
print(f"Current day or night: {is_day}")
print(f"Current precipitation: {current_precipitation}mm")
print(f"Current weather code: {current_weather_code:.0f}")
print(f"Current weather description: {weather_description}")

America/New_York
Coordinates: 40.74°N -73.87°E
Elevation: 18.0m asl
Timezone: GMT-4

Current date: 2025-08-08
Current local time: 14:30 EDT
Current universal time: 18:30 UTC
Current temperature_2m: 27.52°C
Current apparent_temperature: 27.44°C
Current day or night: day
Current precipitation: 0.0mm
Current weather code: 0
Current weather description: Clear sky


In [ ]:
# storing and updating the current data in a spreadsheet

def log_current(response, csv_path="dp-current-1.csv"):
    
    # place data
    tz_name = response.Timezone().decode("utf-8")
    local_tz = ZoneInfo(tz_name)
    latitude = round(response.Latitude(), 4),
    longitude = round(response.Longitude(), 4),
    elevation = response.Elevation(),
    timezone = response.TimezoneAbbreviation().decode('utf-8')

    # current data
    current = response.Current()
    ts_utc = int(current.Time())

    row = {
        "Place": tz_name,
        "Latitude": latitude,
        "Longitude": longitude,
        "Elevation": elevation,
        "Timezone": timezone,
        
        "timestamp_utc": pd.to_datetime(ts_utc, unit="s", utc=True),
        "date_local": datetime.fromtimestamp(ts_utc, tz=local_tz).strftime("%Y-%m-%d"),
        "time_local": datetime.fromtimestamp(ts_utc, tz=local_tz).strftime("%H:%M %Z"),
        "time_utc": datetime.fromtimestamp(ts_utc, tz=timezone.utc).strftime("%H:%M UTC"),

        "temperature_2m_C": float(current.Variables(0).Value()),
        "apparent_temperature_C": float(current.Variables(1).Value()),
        "is_day": "day" if int(current.Variables(2).Value()) == 1 else "night",
        "precipitation_mm": float(current.Variables(3).Value()),
        "weather_code": int(current.Variables(4).Value()),
    }


NameError: name 'ts_utc' is not defined

In [48]:
# process hourly data

hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_apparent_temperature = hourly.Variables(1).ValuesAsNumpy()
hourly_precipitation = hourly.Variables(2).ValuesAsNumpy()
hourly_cloud_cover = hourly.Variables(3).ValuesAsNumpy()
hourly_is_day = hourly.Variables(4).ValuesAsNumpy()
hourly_sunshine_duration = hourly.Variables(5).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["apparent_temperature"] = hourly_apparent_temperature
hourly_data["precipitation"] = hourly_precipitation
hourly_data["cloud_cover"] = hourly_cloud_cover
hourly_data["is_day"] = hourly_is_day
hourly_data["sunshine_duration"] = hourly_sunshine_duration

hourly_dataframe = pd.DataFrame(data = hourly_data)
print("\nHourly data\n", hourly_dataframe)



Hourly data
                          date  temperature_2m  apparent_temperature  \
0   2025-08-01 04:00:00+00:00       19.217501             19.671045   
1   2025-08-01 05:00:00+00:00       18.917501             19.164619   
2   2025-08-01 06:00:00+00:00       18.517500             18.191877   
3   2025-08-01 07:00:00+00:00       18.217501             18.086433   
4   2025-08-01 08:00:00+00:00       18.067501             17.626640   
..                        ...             ...                   ...   
331 2025-08-14 23:00:00+00:00       28.224001             29.294258   
332 2025-08-15 00:00:00+00:00       27.324001             28.631718   
333 2025-08-15 01:00:00+00:00       26.974001             28.509171   
334 2025-08-15 02:00:00+00:00       26.874001             28.694611   
335 2025-08-15 03:00:00+00:00       26.724001             28.820423   

     precipitation  cloud_cover  is_day  sunshine_duration  
0              0.0        100.0     0.0           0.000000  
1          

In [12]:
# process daily data

daily = response.Daily()
daily_weather_code = daily.Variables(0).ValuesAsNumpy()
daily_sunrise = daily.Variables(1).ValuesInt64AsNumpy()
daily_sunset = daily.Variables(2).ValuesInt64AsNumpy()
daily_daylight_duration = daily.Variables(3).ValuesAsNumpy()
daily_sunshine_duration = daily.Variables(4).ValuesAsNumpy()
daily_precipitation_sum = daily.Variables(5).ValuesAsNumpy()
daily_precipitation_probability_max = daily.Variables(6).ValuesAsNumpy()
daily_apparent_temperature_max = daily.Variables(7).ValuesAsNumpy()
daily_apparent_temperature_min = daily.Variables(8).ValuesAsNumpy()
daily_temperature_2m_max = daily.Variables(9).ValuesAsNumpy()
daily_temperature_2m_min = daily.Variables(10).ValuesAsNumpy()
daily_precipitation_hours = daily.Variables(11).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}

daily_data["weather_code"] = daily_weather_code
daily_data["sunrise"] = daily_sunrise
daily_data["sunset"] = daily_sunset
daily_data["daylight_duration"] = daily_daylight_duration
daily_data["sunshine_duration"] = daily_sunshine_duration
daily_data["precipitation_sum"] = daily_precipitation_sum
daily_data["precipitation_probability_max"] = daily_precipitation_probability_max
daily_data["apparent_temperature_max"] = daily_apparent_temperature_max
daily_data["apparent_temperature_min"] = daily_apparent_temperature_min
daily_data["temperature_2m_max"] = daily_temperature_2m_max
daily_data["temperature_2m_min"] = daily_temperature_2m_min
daily_data["precipitation_hours"] = daily_precipitation_hours

daily_dataframe = pd.DataFrame(data = daily_data)
print("\nDaily data\n", daily_dataframe)



Daily data
                         date  weather_code     sunrise      sunset  \
0  2025-08-01 04:00:00+00:00          51.0  1754041957  1754093456   
1  2025-08-02 04:00:00+00:00           3.0  1754128415  1754179789   
2  2025-08-03 04:00:00+00:00           2.0  1754214873  1754266120   
3  2025-08-04 04:00:00+00:00           2.0  1754301331  1754352450   
4  2025-08-05 04:00:00+00:00           3.0  1754387790  1754438779   
5  2025-08-06 04:00:00+00:00          51.0  1754474249  1754525107   
6  2025-08-07 04:00:00+00:00          53.0  1754560708  1754611433   
7  2025-08-08 04:00:00+00:00           0.0  1754647167  1754697758   
8  2025-08-09 04:00:00+00:00           0.0  1754733626  1754784082   
9  2025-08-10 04:00:00+00:00           3.0  1754820085  1754870405   
10 2025-08-11 04:00:00+00:00           3.0  1754906545  1754956727   
11 2025-08-12 04:00:00+00:00           3.0  1754993004  1755043048   
12 2025-08-13 04:00:00+00:00           2.0  1755079462  1755129369   
13 2025

In [49]:
hourly_dataframe.to_csv('hourly_dataframe.csv', index=False)
daily_dataframe.to_csv('daily_dataframe.csv', index=False)